In [46]:
from selenium import webdriver
from bs4 import BeautifulSoup
import requests
import re
import collections
import pickle
import time
import random

In [2]:
URL = 'https://birdsoftheworld.org/bow/specieslist'
#URL = 'https://birdsoftheworld-org.ezproxy.library.wur.nl/bow/specieslist'
page = requests.get(URL)
soup = BeautifulSoup(page.content, 'html.parser')

BOWlist = soup.find_all('a')

URL = 'https://birdsoftheworld.org/bow/specieslist'
#URL = 'https://birdsoftheworld-org.ezproxy.library.wur.nl/bow/specieslist'
page = requests.get(URL)
soup = BeautifulSoup(page.content, 'html.parser')

# Create a list with links
BOWpages = ['https://birdsoftheworld.org' + pages.get('href').replace('introduction', '') for pages in BOWlist 
                   if pages.get('href') != None
                   if pages.get('href').startswith('/bow/species')]

In [4]:
# Init driver
browser = webdriver.Safari()

In [5]:
# Login URL
browser.get("https://secure.birds.cornell.edu/cassso/login?service=https%3A%2F%2Fbirdsoftheworld.org%2Flogin%2Fcas")

# Username
username = browser.find_element_by_id("input-user-name")
username.clear()
username.send_keys("RobertvdV")

# Password
password = browser.find_element_by_name("password")
password.clear()
password.send_keys("XXXXXXXXXX")

# Sign in
SignIn = browser.find_element_by_id("form-submit")
SignIn.send_keys(u'\ue007')

In [37]:
browser.get(BOWpages[10566] + 'appearance')
SleepTime = random.randint(2, 6)
time.sleep(SleepTime)


In [44]:
FreeBirds = [2838, 3100, 10566, 2534, 3661]

In [38]:
HTML = browser.page_source
soup = BeautifulSoup(HTML, 'html.parser')

In [66]:
# Init dict
DataBOW = collections.defaultdict(list)

# Reference Remover
ReferenceRemover1 = ' \(\d+\n\t.+?\n\t\n\t\tClose\n\t\n\)'
ReferenceRemover2 = ' \[\d+\n\t.+?\n\t\n\t\tClose\n\t\n\]'
# Brackets Remover
BracketsRemover = ' \((.*?)\)'

# Loop over the test set
for BirdIDs in FreeBirds:
    
    browser.get(BOWpages[BirdIDs] + 'appearance')
    SleepTime = random.randint(2, 6)
    time.sleep(SleepTime)
    
    HTML = browser.page_source
    soup = BeautifulSoup(HTML, 'html.parser')

    # Bird name 
    if len(soup.title.text.split(' - ')) == 4:

        BirdName = soup.title\
                        .text\
                        .split(' - ')[1]\
                        .lstrip()
    else:
        BirdName = soup.title\
                    .text\
                    .split(' - ')[0]\
                    .lstrip()

    # Find all text data
    Paragraphs = soup.find_all('p')

    # Loop over the text
    for Para in Paragraphs[1:]:
        try:

            # Check to which section the text belongs
            SubSection = Para.find_previous_siblings()[0]\
                             .text\
                             .rstrip()\
                             .lstrip()

            # Check to which section the text belongs
            SubSubSection = Para.find_previous_siblings()[-1]\
                                 .text\
                                 .rstrip()\
                                 .lstrip()

        # If a text does not have a header, skip    
        except:
            continue

        if 'Formative' in SubSection:
            continue
        if 'Juvenile' in SubSection:
            continue
        if 'Immature' in SubSection:
            continue

        # See which are needed and append with 1
        if SubSection == 'Identification' or\
           SubSection == 'Adult' or\
           SubSubSection == 'Plumages' or\
           SubSubSection == 'Bare Parts':

            # Clean the text from references (NEEDS EDIT)    
            TextCleaned = re.sub(ReferenceRemover1, '', Para.text, re.DOTALL)
            TextCleaned = re.sub(ReferenceRemover2, '', TextCleaned, re.DOTALL)
            TextCleaned = re.sub(BracketsRemover, '', TextCleaned)

            # Drop useless short nonsense
            if len(TextCleaned.split()) <2:
                continue
            else:
                DataBOW[BirdName].append(tuple([1, TextCleaned]))

        # Otherwise label 2        
        else:
            # Clean the text from references (NEEDS EDIT)    
            TextCleaned = re.sub(ReferenceRemover1, '', Para.text, re.DOTALL)
            TextCleaned = re.sub(ReferenceRemover2, '', TextCleaned, re.DOTALL)
            TextCleaned = re.sub(BracketsRemover, '', TextCleaned)

            # Drop useless short nonsense
            if len(TextCleaned.split()) <2:
                continue
            else:
                DataBOW[BirdName].append(tuple([0, TextCleaned]))

In [67]:
DataBOW[BirdName]

[(0,
  'You are currently viewing one of the free accounts available in our complimentary tour of Birds of the World. In this courtesy review, you can access all the life history articles and the multimedia galleries associated with this account.'),
 (0, 'For complete access to all accounts, a subscription is required.'),
 (0, 'Subscribe Now'),
 (1,
  'Medium-sized barbet, smaller than Brown-headed Barbet, with which it overlaps, and from which it easily is distinguished by its conspicuous white cheek-stripe. Mostly a deep green color, brighter on the underparts. Crown is dark brown, with black between the base of the bill and the eye. Short white supercilium, and a broad white cheek patch that gives the species its English name. Throat whitish; the breast is brown and streaked with white. The bill is relatively short compared to that of Brown-headed Barbet, and is brownish pink in color, with prominent black bristles projecting forward from the base (1\n\tAli, S., and S. D. Ripley Han

In [68]:
with open('dataBOW.pkl', 'wb') as f:
    pickle.dump(DataBOW, f)

In [60]:
(1, TextCleaned)

(1, '')

In [61]:
test = []

In [62]:
test += (1, TextCleaned)

In [63]:
test

[1, '']